In [56]:
# imports
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import time
import boto3
from urllib.request import urlopen
import json
import ffmpeg
import pyworld
from scipy.io import wavfile
import subprocess
from tqdm import tqdm

In [48]:
# AWS Constants
S3_BUCKET_NAME = 'inflectionbucket'
AWS_PROFILE_NAME = 'patwang123'
AWS_REGION = 'us-east-1'

In [49]:
# Other constants
SUPPORTED_FORMATS = ['.wav', '.mp3']

human_folder = "human"
joanna_folder = "joanna"
transcription_folder = "transcriptions"

folders = [human_folder, joanna_folder, transcription_folder]
    
recording_uri = lambda u: f"{human_folder}/{u}" # our human voices to be tra
data_uri = lambda u: f"{joanna_folder}/{u}"
transcription_uri = lambda u: f"transcriptions/{u}"
s3_recording_uri = lambda u: f"s3://{S3_BUCKET_NAME}/{recording_uri(u)}"

In [50]:
# Create folders if does not already exist
# Recording -- our voices that are being converted to text and then to polly
# Transcriptions -- the transcripts
# Data -- the voice generated results from AWS
cwd = os.getcwd()
for folder in folders:
    path = os.path.join(cwd, folder)
    if not os.path.exists(path):
        os.makedirs(path)

In [51]:
# Start s3, Polly, and Transcribe AWS sessions
session = boto3.Session(profile_name=AWS_PROFILE_NAME, region_name=AWS_REGION)
s3 = session.client('s3')
polly = session.client("polly")
transcribe = session.client('transcribe')

In [52]:
# Create bucket if does not exist yet
try:
    s3.create_bucket(Bucket=S3_BUCKET_NAME)
except:
    pass

In [58]:
# to check if we have already processed it, as if it is already in data, there is no need to process it again
data_done = set(os.listdir(os.path.join(cwd, joanna_folder)))

# list of file names that should be located in the recordings/ to process
filenames = set(os.listdir(os.path.join(cwd, human_folder))).difference(data_done)

in_progress = list(filenames)
completed = []
tot = len(in_progress)
errors = []

print(f"{tot} files to process: {in_progress}")

if not tot:
    assert False, "No files left to process"

print("Uploading to s3")
# Uploads all files into s3
for filename in tqdm(filenames):
    with closing(open(recording_uri(filename), 'rb')) as f:
        p = s3.upload_fileobj(f, S3_BUCKET_NAME, recording_uri(filename))

print("Done uploaded to s3")

print("Transcribing...")
print("Starting transcription jobs...")
for f in in_progress:
    job_name = f
    job_uri = s3_recording_uri(f)
    try:
        transcribe.start_transcription_job(
            TranscriptionJobName=job_name,
            Media={'MediaFileUri': job_uri},
            MediaFormat="wav",
            LanguageCode='en-US'
        )
    except:
        transcribe.delete_transcription_job(TranscriptionJobName=job_name)
        transcribe.start_transcription_job(
            TranscriptionJobName=job_name,
            Media={'MediaFileUri': job_uri},
            MediaFormat="wav",
            LanguageCode='en-US'
        )
    
print("Started all transcription jobs")
while True:
    incomplete = []
    for f in in_progress:
        status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            if status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
                completed.append(f)
            else:
                error.append(f)
        else:
            incomplete.append(f)
    in_progress = incomplete
    print(f"{(len(completed)/tot * 10000) // 100}% of the way done transcribing.")
    if not in_progress:
        break
    time.sleep(15)

print("Done transcribing")

if error:
    print(f"The following have errored: {error}")
else:
    print("All transcribed successfully")
print("Getting the joanna voice shenanigans")
for f in tqdm(completed):
    job_name = f
    job_uri = s3_recording_uri(f)
    # Get location of data from transcription job and read in json
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    transcribe_url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
    opened = urlopen(transcribe_url)
    data_json = json.loads(opened.read())

    # Delete transcription job to clean up AWS or whatever
    transcribe.delete_transcription_job(TranscriptionJobName=job_name)

    # Save file to transcription folder
    with closing(open(transcription_uri(f"{os.path.splitext(filename)[0]}.json"), 'w')) as s:
        json.dump(data_json, s)

    # Using polly to synthesize speech based on transcript
    try:
        # Request speech synthesis
        response = polly.synthesize_speech(Text=data_json['results']['transcripts'][0]['transcript'],
                                           OutputFormat="mp3",
                                           VoiceId="Joanna")
    except (BotoCoreError, ClientError) as error:
        # The service returned an error, exit gracefully
        print(error)
        continue

    # Save polly response to data
    if 'AudioStream' in response:
        temp = f + "TEMP"
        with closing(response["AudioStream"]) as stream:
            save_path = os.path.join(os.getcwd(), joanna_folder, temp)
            file = open(save_path, 'wb')
            file.write(stream.read())
        command = f"ffmpeg -i {data_uri(temp)} {data_uri(f)} -y"
        # could multithread if you really want to be sweaty but whatever
        subprocess.call(command,shell=True,stdout=subprocess.DEVNULL,stderr=subprocess.STDOUT) 
        os.remove(save_path)
print("Done synthesizing")

119 files to process: ['trump_23.wav', 'trump_15.wav', 'trump_47.wav', 'trump_0.wav', 'trump_82.wav', 'trump_35.wav', 'trump_111.wav', 'trump_49.wav', 'trump_114.wav', 'trump_69.wav', 'trump_36.wav', 'trump_65.wav', 'trump_38.wav', 'trump_30.wav', 'trump_41.wav', 'trump_84.wav', 'trump_56.wav', 'trump_63.wav', 'trump_4.wav', 'trump_43.wav', 'trump_27.wav', 'trump_115.wav', 'trump_48.wav', 'trump_85.wav', 'trump_99.wav', 'trump_16.wav', 'trump_39.wav', 'trump_28.wav', 'trump_112.wav', 'trump_75.wav', 'trump_98.wav', 'trump_37.wav', 'trump_14.wav', 'trump_90.wav', 'trump_42.wav', 'trump_55.wav', 'trump_58.wav', 'trump_52.wav', 'trump_29.wav', 'trump_100.wav', 'trump_25.wav', 'trump_113.wav', 'trump_88.wav', 'trump_61.wav', 'trump_106.wav', 'trump_2.wav', 'trump_93.wav', 'trump_5.wav', 'trump_34.wav', 'trump_40.wav', 'trump_94.wav', 'trump_31.wav', 'trump_95.wav', 'trump_83.wav', 'trump_13.wav', 'trump_18.wav', 'trump_103.wav', 'trump_67.wav', 'obama_0.wav', 'trump_101.wav', 'trump_107.wa

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:34<00:00,  3.46it/s]


Done uploaded to s3
Transcribing...
0.0% of the way done transcribing.
100.0% of the way done transcribing.
Done transcribing
['trump_23.wav', 'trump_15.wav', 'trump_47.wav', 'trump_0.wav', 'trump_82.wav', 'trump_35.wav', 'trump_111.wav', 'trump_49.wav', 'trump_114.wav', 'trump_69.wav', 'trump_36.wav', 'trump_65.wav', 'trump_38.wav', 'trump_30.wav', 'trump_41.wav', 'trump_84.wav', 'trump_56.wav', 'trump_63.wav', 'trump_4.wav', 'trump_43.wav', 'trump_27.wav', 'trump_115.wav', 'trump_48.wav', 'trump_85.wav', 'trump_99.wav', 'trump_16.wav', 'trump_39.wav', 'trump_28.wav', 'trump_112.wav', 'trump_75.wav', 'trump_98.wav', 'trump_37.wav', 'trump_14.wav', 'trump_90.wav', 'trump_42.wav', 'trump_55.wav', 'trump_58.wav', 'trump_52.wav', 'trump_29.wav', 'trump_100.wav', 'trump_25.wav', 'trump_113.wav', 'trump_88.wav', 'trump_61.wav', 'trump_106.wav', 'trump_2.wav', 'trump_93.wav', 'trump_5.wav', 'trump_34.wav', 'trump_40.wav', 'trump_94.wav', 'trump_31.wav', 'trump_95.wav', 'trump_83.wav', 'trump

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

...

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

Done synthesizing


ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l